In [ ]:
import requests
import openai
import os
import pandas as pd
from pprint import pprint

In [ ]:
# In a Jupyter cell, run:
import sys
print(sys.executable)

In [ ]:
base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
client = openai.OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url=base_url
)

In [ ]:
models = client.models.list()
df = pd.DataFrame([
    {'name': m.id, 'display_name': m.display_name} 
    for m in models
])
display(df)

In [105]:
response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {
            "role": "user",
            "content": (
                "Explain how AI works.\n\n"
                "First, provide a brief reasoning summary explaining the key ideas "
                "Do NOT include planning text, meta commentary, or role-played thoughts.Only provide a brief reasoning summary and a final answer."
                "at a high level (not internal chain-of-thought).\n\n"
                "Format the response exactly as:\n"
                "<THOUGHT_SUMMARY>...</THOUGHT_SUMMARY>\n"
                "<FINAL>...</FINAL>"
            )
        }
    ],
    extra_body={
        "extra_body":{
        "google": {
            "thinking_config": {
                "include_thoughts": True
            }
        }
    }
    }
)

print(response.choices[0].message.content)

<thought>**How AI Works: My Perspective**

Okay, so I'm thinking about how to explain AI to someone who already *gets* the big picture, but maybe needs a more concrete breakdown of the mechanics. My plan is simple: let's distill the core idea of AI down to its essence.

First, I've got to highlight that AI is essentially about simulating intelligence through algorithms and data. It's about letting the machine "learn," rather than explicitly programming every scenario.

My reasoning starts with these key points:

*   AI is all about *emulating* human intelligence.
*   It hinges on algorithms and data working together.
*   Machine learning is the pivotal element, learning from patterns.
*   These are used to build statistical models that predict or make decisions.
*   Deep learning takes it up a notch with complex neural networks, and
*   The overall goal is to perform intelligent tasks.

My goal is to provide a concise, expert-level overview. Ultimately, I need to convey how AI uses vas

In [106]:
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<thought>**How AI Works: My Perspective**\n\nOkay, so I\'m thinking about how to explain AI to someone who already *gets* the big picture, but maybe needs a more concrete breakdown of the mechanics. My plan is simple: let\'s distill the core idea of AI down to its essence.\n\nFirst, I\'ve got to highlight that AI is essentially about simulating intelligence through algorithms and data. It\'s about letting the machine "learn," rather than explicitly programming every scenario.\n\nMy reasoning starts with these key points:\n\n*   AI is all about *emulating* human intelligence.\n*   It hinges on algorithms and data working together.\n*   Machine learning is the pivotal element, learning from patterns.\n*   These are used to build statistical models that predict or make decisions.\n*   Deep learning takes it up a notch with complex neural networks, and\n*   The overall goal is to perform intelligent

In [ ]:
import re
from typing import Dict, Optional

def structured_output(llm_response: str) -> Dict[str, Optional[str]]:
    def extract(tag: str) -> Optional[str]:
        pattern = rf"<{tag}>(.*?)</{tag}>"
        match = re.search(pattern, llm_response, re.S | re.I)
        return match.group(1).strip() if match else None
    
    return {
        "thought": extract("thought"),               # optional, unstable
        "thought_summary": extract("THOUGHT_SUMMARY"),  # preferred
        "final_answer": extract("FINAL"),             # required
    }
out = structured_output(response.choices[0].message.content)
if not out["final_answer"]:
    raise ValueError("No <FINAL> section found in LLM output")
pprint(structured_output(response.choices[0].message.content))

In [ ]:
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='', flush=True)

In [ ]:
from nsepython import nsefetch

def fetch_price(symbol: str):
    '''
    Get the current price of a security, just pass in the ticker symbol.
    '''
    try:
        # Get quote from NSE directly
        url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol}'
        data = nsefetch(url)
        
        price,percent_change = data['priceInfo']['lastPrice'],data['priceInfo']['pChange']
        return round(price, 2),round(percent_change,2),data
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "fetch_price",
      "description": "Get the current price of a security, just pass in the ticker symbol.",
      "parameters": {
        "type": "object",
        "properties": {
          "symbol": {
            "type": "string",
            "description": "The ticker symbol of the security being fetched, just pass in the ticker symbol and not the full name e.g. INFY, RELIANCE ",
          }
        },
        "required": ["symbol"],
      },
    }
  }
]

In [ ]:
from pydantic import BaseModel, Field

class GetFetchPriceArgs(BaseModel):
    symbol: str = Field(...,description="The ticker symbol of the security being fetched, just pass in the ticker symbol and not the full name e.g. INFY, RELIANCE ")

schema = [{
    'type':'function',
    "function":{
        "name":"fetch_price",
        "description":"Get the current price of a security, just pass in the ticker symbol.",
        "parameters": GetFetchPriceArgs.model_json_schema()
    }
}
]

In [113]:
response = response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {
            "role": "user",
            "content": (
                "What is the stock price of hdfc bank?"
            )
        }
    ],
    tools= schema,         #type: ignore
    tool_choice= 'auto'
)

print(response.choices[0].message.content)

None


In [118]:
pprint(response.choices[0].message.tool_calls)

[ChatCompletionMessageFunctionToolCall(id='function-call-11965070616874333571', function=Function(arguments='{"symbol":"HDFCBANK"}', name='fetch_price'), type='function')]


In [149]:
pprint(response.choices[0].message.tool_calls[0].function.arguments)

'{"symbol":"HDFCBANK"}'


In [153]:
from typing import Any
import json

class Agent:
    def __init__(self, client,system:str="",tools:list = None) -> None :
        self.client = client
        self.system = system
        self.messages : list = []
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self,message = "", *args: Any, **kwds: Any) -> Any:
        if message:
            self.messages.append({"role": "user", "content": message})
        
        final_assistant_content = self.execute()
        
        if message:
            self.messages.append({"role": "assistant", "content": final_assistant_content})

        return final_assistant_content
    
    def execute(self):
        while True:
            completion = self.client.chat.completions.create(
                model="gemini-2.5-flash",
                messages = self.messages,
                tools = self.tools,
                tool_choice = "auto"
            )
            response_message = completion.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []

                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)

                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        exected_output = function_to_call(**function_args)
                        print(exected_output)
                        tool_output_content = str(exected_output)
                        print(f"Executing tool:{function_name} with args {function_args}, Output:{tool_output_content[:500]}") #For Debugging

                        tool_outputs.append(
                            {
                                "tool_call_id":tool_call.id,
                                "role":"tool",
                                "name":function_name,
                                "content":tool_output_content
                            }
                        )
                self.messages.extend(tool_outputs)
            
            else:
                return response_message.content
        

In [154]:
client = openai.OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url=base_url
)
system  = "You are helpful AI assistant."
tools = schema
agent = Agent(client, system, tools)

In [155]:
agent("What is the price of Infosys stock?")

'\n'